In [ ]:
#load sql
%load_ext sql
%sql sqlite://

'Connected: @None'

In [ ]:
#importing pandas
import pandas as pd

In [ ]:
# Loading our grand electors dataset  from the respective CSV file 
with open('/content/GrandElectors_by_state.csv','r') as f:
    GrandElectors_by_state = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql DROP TABLE if EXISTS GrandElectors_by_state;
%sql PERSIST GrandElectors_by_state;
%sql SELECT * FROM GrandElectors_by_state LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,GrandElectors
Alabama,9
Alaska,3
Arizona,11
Arkansas,6
California,55


In [ ]:
# Loading our population by state dataset  from the respective CSV file 
with open('/content/Population_by_state.csv','r') as f:
    Population_by_state = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql DROP TABLE if EXISTS Population_by_state;
%sql PERSIST Population_by_state;
%sql SELECT * FROM Population_by_state LIMIT 5;


 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,Population
CALIFORNIA,39144818
TEXAS,27469114
FLORIDA,20271272
NEW YORK,19795791
ILLINOIS,12859995


In [ ]:
#order by decending order of grand electors
%%sql
select State, GrandElectors
from GrandElectors_by_state
order by GrandElectors desc;


In [ ]:
#target all the states at the top of the list until the cumulative sum of 
#grand electors won is larger than half the total number of Grand Electors in the country.
%%sql
select GrandElectors, State
    GrandElectors,
    (SELECT Sum(GrandElectors)/2 FROM GrandElectors_by_state) #here i divided by two to get the average needed but remove the /2 it will give the running total
FROM GrandElectors_by_state
group by GrandElectors


In [ ]:
#capitalise
%%sql
update GrandElectors_by_state
set 
State = upper(State)

 * sqlite://
51 rows affected.


[]

In [ ]:
%sql SELECT * FROM GrandElectors_by_state LIMIT 5;

 * sqlite://
Done.


State,GrandElectors
ALABAMA,9
ALASKA,3
ARIZONA,11
ARKANSAS,6
CALIFORNIA,55


In [ ]:
# join the 2 tables
%%sql
SELECT GrandElectors_by_state.State,Population,GrandElectors 
FROM GrandElectors_by_state INNER JOIN Population_by_state 
ON GrandElectors_by_state.State = Population_by_state.State LIMIT 5;


In [ ]:
#change the name of the "District of Columbia" state to its short version "DC"
%%sql
UPDATE GrandElectors_by_state
SET State = "DC" 
WHERE State = "DISTRICT OF COLUMBIA";
SELECT * FROM GrandElectors_by_state;

In [ ]:
#compute the ratio between the number of grand electors and the population.
#Please create a new column with that ratio.
%%sql
ALter table GrandElectors_by_state
add Ratio varchar  (255);
Select GrandElectors, (Count(GrandElectors)* 100 / (Select Count(Population) From Population_by_state)) as Ratio
From GrandElectors_by_state
Group By GrandElectors

In [ ]:
# To order the states by decreasing ratio of Grand Electors per capita. That will make our priority list.
%%sql
SELECT State, Ratio
FROM GrandElectors_by_state
order by  Ratio desc;

In [ ]:
#To compute the running total of Grand Electors in that sorted list.
%%sql
select GrandElectors, State
    GrandElectors,
    (SELECT Sum(GrandElectors)/2 FROM GrandElectors_by_state)
FROM GrandElectors_by_state
group by GrandElectors

 * sqlite://
Done.


GrandElectors,GrandElectors_1,(SELECT Sum(GrandElectors)/2 FROM GrandElectors_by_state)
3,WYOMING,269
4,RHODE ISLAND,269
5,WEST VIRGINIA,269
6,UTAH,269
7,OREGON,269
8,LOUISIANA,269
9,SOUTH CAROLINA,269
10,WISCONSIN,269
11,TENNESSEE,269
12,WASHINGTON,269


In [ ]:
#Independently, to compute the half of the total of Grand Electors overall (in the whole country):
%%sql
select avg(GrandElectors)
from GrandElectors_by_state;

 * sqlite://
Done.


avg(GrandElectors)
10.549019607843137


In [ ]:
# To filter our sorted list of states in order to keep only the (top)
# ones enabling us to reach the computed threshold.
%%sql
SELECT State,GrandElectors
from GrandElectors_by_state 
where GrandElectors > 10 ;

 * sqlite://
Done.


State,GrandElectors
ARIZONA,11
CALIFORNIA,55
FLORIDA,29
GEORGIA,16
ILLINOIS,20
INDIANA,11
MASSACHUSETTS,11
MICHIGAN,16
NEW JERSEY,14
NEW YORK,29
